In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tl_class_dict = {
 0: "Face Brick",
 1: "Timber",
 2: "Steel Sheet",
 3: "Plastered and Painted",
 4: "Concrete and Glass",
 5: "Glass",
 6: "Fiber Cement Sheet",
 7: "Concrete Panels",
 8: "Unidentified"
}

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Assuming you have the same model architecture as before
model_ft = models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(tl_class_dict))

# Specify the path to your fine-tuned model
model_path = 'best Model/finalModel.pt'

# Load the state dictionary
model_ft.load_state_dict(torch.load(model_path))

# Make sure to set the model to evaluation mode
model_ft.eval()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
columns = [
    "Face Brick",
    "Timber",
    "Steel Sheet",
    "Plastered and Painted",
    "Concrete and Glass",
    "Glass",
    "Fiber Cement Sheet",
    "Concrete Panels",
    "Unidentified"
]

In [ ]:
import os
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image

def extract_object_id(img_name):
    return img_name.split('_')[0]

def create_prediction_column_batched(image_dir, csv_path, model, columns, output_csv_path, batch_size):
    df = pd.read_csv(csv_path)

    if 'Type' in df.columns:
        df = df.rename(columns={'Type': 'Typology'})
        print('Renamed')

    model.to(device)
    model.eval()

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create a dictionary to store image paths for each object ID
    object_id_to_path = {str(img_id): [] for img_id in df['OBJECTID *']}
    # Populate the dictionary with image paths
    for img_name in os.listdir(image_dir):
        img_id = extract_object_id(img_name)
        object_id_to_path[img_id].append(os.path.join(image_dir, img_name))
    predictions = []

    total_images = len(df)
    num_batches = (total_images + batch_size - 1) // batch_size

    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, total_images)

        batch_df = df.iloc[start_idx:end_idx]

        batch_predictions = []

        for index, row in batch_df.iterrows():
            img_id = str(int(row['OBJECTID *']))
            img_paths = object_id_to_path.get(img_id, [])
            if img_paths:
                img_path = img_paths[0]  # Take the first image path

                try:
                    img = Image.open(img_path).convert("RGB")
                    img = transform(img).unsqueeze(0)
                    img = img.to(device)

                    with torch.no_grad():
                        output = model(img)
                        _, pred = torch.max(output, 1)
                        predicted_class = columns[pred.item()]
                        batch_predictions.append(predicted_class)
                except Exception as e:
                    # print(f"Error processing image {img_path}: {e}")
                    batch_predictions.append("Error")
            else:
                # print(f"No matching image found for object ID {img_id}")
                batch_predictions.append("No Match")

        predictions.extend(batch_predictions)

        print(f"Processed batch {batch_num + 1}/{num_batches}, Images processed: {end_idx}/{total_images}")

    df['prediction'] = predictions
    df = df[['OBJECTID *', 'Typology', 'prediction']]
    df.to_csv(output_csv_path, index=False)


In [ ]:
#Example_use
image_dir = "/content/drive/MyDrive/F0_greaterthan3.5lessthan4/images"  #your image directory
csv_path = "/content/drive/MyDrive/F0_greaterthan3.5lessthan4/F0_greaterorequal3.5_lessthan4.csv"  #your dataset CSV file
output_csv_path = "/content/drive/MyDrive/Predictions/F0_Greater3.5_less4.0_prediction.csv"

create_prediction_column_batched(image_dir, csv_path, model_ft, columns, output_csv_path, batch_size=200)